# Computational Neuroscience Final Project
## Elizabeth Frey

This project... add description...

[English to French translation Dataset](https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench/)

[Tutorial](https://pythonalgos.com/2022/01/02/build-a-gru-rnn-in-keras/)

## Set-Up

In [3]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Parse Dataset

In [4]:
import pandas as pd

In [6]:
# read the csv file and create a dataframe
df = pd.read_csv('eng-french.csv')

## Train Model

## Test Model

## Analyze Model